In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
data=pd.read_csv("/kaggle/input/bank-marketing-dataset/bank.csv")
data.head()

In [ ]:
data.describe()

Okay, I think balance and deposit have strong negative correlation. 

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
data["deposit_numeric"]=le.fit_transform(data["deposit"])

In [ ]:
data[["balance","deposit_numeric"]].corr()

Interesting correlation is weak. Maybe outlier values effect it's too much.

In [ ]:
data_new=data[(data["balance"]>122) & (data["balance"]>2000)]
data_new[["balance","deposit_numeric"]].corr()

## Even Smaller. Maybe if balance is negative all say yes to deposit.

In [ ]:
data_new=data[(data["balance"]<-100)]
data_new["deposit"].value_counts()

Okay we see more clear observation. İf balance is negative it's more likely to say no to deposit. 

## Okay what about balance with age?

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,12))
sns.relplot(data=data,x="age",y="balance",hue="deposit")

Okay can't clearly separable also with age and balance variable.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,12))
sns.relplot(data=data,x="balance",y="campaign",hue="deposit")

Okay also can't clearly separable. 

Okay now we see all correlations.

In [ ]:
plt.figure(figsize=(20,16))
sns.heatmap(data.corr(),annot=True)

Deposit numeric has strong correlation with duration it's interesting. 

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,12))
sns.relplot(data=data,x="balance",y="duration",hue="deposit")

Okay this 2 is more separable than others. Hovewer it isn't perfect.

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
plt.figure(figsize=(15,12))
sns.relplot(data=data,x="age",y="duration",hue="deposit")

## Okay with these 3 feature (Age, Balance and Duration) we will have good results. 

In [ ]:
x=data[["age","duration","balance"]]
y=data["deposit_numeric"]

from sklearn.naive_bayes import GaussianNB
naive_bayes=GaussianNB()
from sklearn.model_selection import cross_val_score
scores_naive = cross_val_score(naive_bayes, x, y, cv=10)
print("Naive Bayes")
scores_naive

Okay naive bayes's result is not stable. Naive Bayes can't work fine with bigger datasets.

In [ ]:
from catboost import CatBoostClassifier


catb = CatBoostClassifier(iterations=1000,verbose=0)

scores_catb = cross_val_score(catb, x, y, cv=10)
print("Catboost")
print(scores_catb)

Worst better than naive bayes but also not that great. Okay we see classical train test split result.

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(iterations=1000,verbose=0)

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

Okay only 3 feature and result is fine. Accuracy is %75.

## With all the features:

In [ ]:

x_full=data.iloc[:,0:16]
xfull_train, xfull_test, y_train, y_test = train_test_split(x_full, y, test_size=0.25, random_state=142)

categorical_features_indices = np.where(x_full.dtypes != np.float)[0]

catb = CatBoostClassifier(iterations=1000,verbose=0,cat_features=categorical_features_indices)

catb.fit(xfull_train,y_train)
ypred=catb.predict(xfull_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

With all features. Accuracy is %85 and confusion matrix also looks just fine. 

In [ ]:


catb = CatBoostClassifier(verbose=0,cat_features=categorical_features_indices)

catb.fit(xfull_train,y_train)
ypred=catb.predict(xfull_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

## Okay, Categorical variables have important informations. Add some Categorical features too our 3 feature model.

In [ ]:
data

Add marital status and loan effected most I think.

In [ ]:
x=data[["age","duration","balance","marital","loan"]]
y=data["deposit_numeric"]


catb = CatBoostClassifier(verbose=0,cat_features=[3,4])

scores_catb = cross_val_score(catb, x, y, cv=10)
print("Catboost")
print(scores_catb)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(verbose=0,cat_features=[3,4])

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

Slightly better maybe job is the most important categorical features.

In [ ]:
x=data[["age","duration","balance","marital","loan","job"]]
y=data["deposit_numeric"]


catb = CatBoostClassifier(verbose=0,cat_features=[3,4,5])

scores_catb = cross_val_score(catb, x, y, cv=10)
print("Catboost")
print(scores_catb)

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(verbose=0,cat_features=[3,4,5])

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

## Now Delete Marital and loan features and only use job categorical variable.

In [ ]:
x=data[["age","duration","balance","job"]]
y=data["deposit_numeric"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(verbose=0,cat_features=[3])

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

Now we add education.

In [ ]:
x=data[["age","duration","balance","job","education"]]
y=data["deposit_numeric"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(verbose=0,cat_features=[3,4])

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

In [ ]:
x=data[["age","duration","balance","job","education","housing"]]
y=data["deposit_numeric"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(verbose=0,cat_features=[3,4,5])

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

In [ ]:
x=data[["age","duration","balance","job","education","housing","marital"]]
y=data["deposit_numeric"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(verbose=0,cat_features=[3,4,5,6])

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

In [ ]:
x=data[["age","duration","balance","campaign","job","education","housing","marital"]]
y=data["deposit_numeric"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(verbose=0,cat_features=[4,5,6,7])

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

In [ ]:
importances=catb.get_feature_importance()
features=["age","duration","balance","campaign","job","education","housing","marital"]
for i in range(0,len(importances)):
    print(features[i],": ",importances[i])

## Duration is the most important featues. First five feature leads all features. Job is more important than education and marital.

Campaign also not that important.

In [ ]:
x=data[["age","duration","balance","pdays","job","education","housing"]]
y=data["deposit_numeric"]

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, random_state=142)
catb = CatBoostClassifier(verbose=0,cat_features=[4,5,6])

catb.fit(x_train,y_train)
ypred=catb.predict(x_test)

import sklearn.metrics as metrik
print(metrik.confusion_matrix(y_true=y_test,y_pred=ypred))
print(metrik.accuracy_score(y_true=y_test,y_pred=ypred))

In [ ]:
importances=catb.get_feature_importance()
features=x.columns
for i in range(0,len(importances)):
    print(features[i],": ",importances[i])

## Pdays also an important column. Duration and Pdays is the most important features. Also with these 7 features we %81.5 accuracy. İt is better than I expected. This model is usable enough.

In [ ]:
catb.save_model("model7features")

## Thanks for Reading :)